<a href="https://colab.research.google.com/github/SugarC21/colab_Open_WebUI/blob/main/Colab_Open_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start Open WebUI

In [ ]:
# @title
name = "" # @param {"type":"string"}
import subprocess
from google.colab import drive

if name == "":
  webui_path = "/content/drive/MyDrive/udocker_exports/open_webui_container.tar";
  ollama_path = "/content/drive/MyDrive/udocker_exports/ollama_container.tar";
else:
  webui_path = "/content/drive/MyDrive/udocker_exports/open_webui_container_" + name + ".tar";
  ollama_path = "/content/drive/MyDrive/udocker_exports/ollama_container_" + name + ".tar";


# Mount Google Drive
drive.mount('/content/drive')



# Install udocker if not installed
!apt-get update
!pip install udocker
!udocker --allow-root install

# Import necessary modules
import os

# Ensure export directory exists
os.makedirs("/content/drive/MyDrive/udocker_exports", exist_ok=True)
!mkdir -p /content/drive/MyDrive/udocker_exports/ollama_models

# Import containers if they exist, otherwise pull and create new ones
if os.path.exists(ollama_path) and os.path.exists(webui_path):
    print("Importing existing containers from Google Drive...")
    subprocess.run(["udocker", "--allow-root", "load", "-i", ollama_path, "ollama_container"])
    subprocess.run(["udocker", "--allow-root", "load", "-i", webui_path, "open_webui_container"])
    !udocker --allow-root create --name=open_webui_container open_webui_container:main
    !udocker --allow-root create --name=ollama_container ollama_container:latest
else:
    print("No saved containers found. Pulling images and creating containers...")
    subprocess.run(["udocker", "--allow-root", "pull", "ollama/ollama"])
    subprocess.run(["udocker", "--allow-root", "pull", "ghcr.io/open-webui/open-webui:main"])

    subprocess.run(["udocker", "--allow-root", "create", "--name=ollama_container", "ollama/ollama"])
    subprocess.run(["udocker", "--allow-root", "create", "--name=open_webui_container", "ghcr.io/open-webui/open-webui:main"])

    # Export containers for future use
    print("Saving containers to Google Drive for future sessions...")
    subprocess.run(["udocker", "--allow-root", "save", "-o", "/content/drive/MyDrive/udocker_exports/ollama_container", "ollama_container"])
    subprocess.run(["udocker", "--allow-root", "save", "-o", "/content/drive/MyDrive/udocker_exports/open_webui_container", "open_webui_container"])

!udocker --allow-root ps

# Set execution mode
#subprocess.run(["udocker", "--allow-root", "setup", "--execmode=P1", "ollama_container"])
#subprocess.run(["udocker", "--allow-root", "setup", "--execmode=P1", "open_webui_container"])
!udocker --allow-root setup --nvidia  ollama_container
!udocker --allow-root setup --nvidia  open_webui_container

# Start the containers as subprocesses and redirect logs to files

# Define log file paths
import time

OLLAMA_LOG = "/content/ollama.log"
WEBUI_LOG = "/content/webui.log"
NGROK_LOG = "/content/ngrok.log"

def stream_logs(log_file):
    with open(log_file, "r") as f:
        f.seek(0, os.SEEK_END)  # Move to the end of the file
        while True:
            line = f.readline()
            if line:
                print(f"{log_file}: {line}", end="")
            time.sleep(0.1)


#!udocker --allow-root run --env=WEBUI_AUTH=False --publish=3000:8080 open_webui_container

# Starting Ollama container with volume mapping for models
print("Starting Ollama container...")
ollama_process = subprocess.Popen(
    [
        "udocker", "--allow-root", "run",
        "--env=OLLAMA_NOPRUNE=true",
        "--publish=11434:11434",
        "--volume=/content/drive/MyDrive/udocker_exports/ollama_models:/root/.ollama",
        "ollama_container"
    ],
    stdout=open(OLLAMA_LOG, "w"),
    stderr=subprocess.STDOUT
)

print("Starting Open-WebUI container...")
webui_process = subprocess.Popen(
    [
        "udocker", "--allow-root", "run",
        "--env=OLLAMA_BASE_URL=http://0.0.0.0:11434",
        "--env=WEBUI_AUTH=False",
        "--publish=3000:8080",
        "open_webui_container"
    ],
    stdout=open(WEBUI_LOG, "w"),
    stderr=subprocess.STDOUT
)

# Check running containers
subprocess.run(["udocker", "--allow-root", "ps"])


In [ ]:
# !udocker --allow-root images -l
# !udocker --allow-root rm 981b744d-b398-3e44-a7bc-6c1cb79d9732

# !curl -v http://0.0.0.0:11434

#webui_process.kill()

!udocker --allow-root inspect -p open_webui_container

In [ ]:
!pip install pyngrok
from pyngrok import ngrok
import time



# Set your ngrok auth token here
from google.colab import userdata
ngrok_auth_token = userdata.get('ngrok_auth_token')
ngrok_auth_token = ngrok_auth_token

# Initialize ngrok with auth token
ngrok_process = subprocess.Popen(
    [
        'ngrok', 'http', '3000',
        '--authtoken', ngrok_auth_token,
        '--log', 'stdout'  # Direct logs to standard output
    ], stdout=open(NGROK_LOG, "w"), stderr=subprocess.STDOUT
)


# Wait for tunnels to establish
time.sleep(30)

# Get public URL using ngrok API
try:
    tunnels = subprocess.check_output(['curl', '-s', 'http://localhost:4040/api/tunnels'])
    public_url = str(tunnels).split('public_url":"')[1].split('"')[0]
    print(f"\nOpen WebUI Public URL: {public_url.replace('http://', 'https://')}\n")
except Exception as e:
    print(f"Error getting ngrok URL: {str(e)}")


import threading

#threading.Thread(target=stream_logs, args=(OLLAMA_LOG,), daemon=True).start()
#threading.Thread(target=stream_logs, args=(WEBUI_LOG,), daemon=True).start()
##threading.Thread(target=stream_logs, args=(NGROK_LOG,), daemon=True).start()

# Show running containers
!udocker --allow-root ps




In [ ]:
# @title Default title text
save_changes = False # @param {"type":"boolean"}
name = "deepseek_70b" # @param {"type":"string"}

if save_changes != True:
  print("here")
  raise SystemExit("Changes not saved!")

# Export ollama container
ollama_container_name = "/content/drive/MyDrive/udocker_exports/ollama_container_" + name + ".tar"
!udocker --allow-root save -o $ollama_container_name ollama_container

# Export open-webui container!
open_webui_container_name = "/content/drive/MyDrive/udocker_exports/open_webui_container_" + name + ".tar"
!udocker --allow-root save -o $open_webui_container_name open_webui_container:main


# Create docker containers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt-get update
!pip install udocker
!udocker --allow-root install

In [ ]:
# Pull necessary container images
!udocker --allow-root pull ollama/ollama
!udocker --allow-root pull ghcr.io/open-webui/open-webui:main

# Create containers from the images
!udocker --allow-root create --name=ollama_container ollama/ollama
!udocker --allow-root create --name=open_webui_container ghcr.io/open-webui/open-webui:main

# Set up execution mode (P1 for best compatibility)
#!udocker --allow-root setup --execmode=P1 ollama_container
#!udocker --allow-root setup --execmode=P1 open_webui_container$


In [ ]:
!mkdir -p /content/drive/MyDrive/udocker_exports  # Ensure export directory exists

!udocker --allow-root images -l
# Export ollama container
!udocker --allow-root save -o /content/drive/MyDrive/udocker_exports/ollama_container.tar ollama/ollama:latest

# Export open-webui container
!udocker --allow-root save -o /content/drive/MyDrive/udocker_exports/open_webui_container.tar ghcr.io/open-webui/open-webui:main


